<a href="https://colab.research.google.com/github/Souviksamanta34/Breast_Cancer_Classification_Using_NN/blob/main/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Load the data from a CSV file
data = pd.read_csv('data_clean_id.csv')

# Define X and y
X = data.drop(columns=['diagnosis', 'id'])
y = data['diagnosis']

# Encode the target variable
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Feature selection using mutual information
selector = SelectKBest(score_func=mutual_info_classif, k=10)  # Adjust k as needed
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)

# Define a neural network classifier
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(np.max(y_train) + 1, activation='softmax')  # Output layer with appropriate units
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, verbose=2)

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, y_pred_classes)
print('Test accuracy:', accuracy)
print(classification_report(y_test, y_pred_classes))

# Save the trained model
model.save('breast_cancer_classifier.h5')
